In [1]:
import warnings
import numpy as np

In [2]:
ffamilies = [
    lambda k: lambda x: x[0, 0]**2 + k * x[0, 0] * x[0, 1] + x[0,1]**2,
]

In [3]:
def partial(f, x, i=[0], dx=1e-6):
    """Computes the i-th partial derivative of f at point x.
    
    Args:
        f: objective function.
        x: point at which partial derivative is computed.
        i: list of coordinates along which derivative is computed (differentiates successively once per coordinate).
        dx: slack for finite difference.
        
    Output:
        (float)
    """
    if not i:
        return f(x)
    x = x.reshape(1, -1)
    h = np.zeros(x.shape)
    h[0, i[0]] = dx
    p1 = partial(f, x + h, i[1:], dx)
    p2 = partial(f, x - h, i[1:], dx)
    return (p1 - p2) / (2*dx)

def deriv1(f, x, i, dx=1e-6):
    return partial(f, x, [i], dx)

def deriv2(f, x, i, j, dx=1e-6):
    return partial(f, x, [i, j], dx)

In [4]:
def gradient(f, x, dx=1e-6):
    """Computes gradient of f at point x.
    
    Args:
        f: objective function.
        x: point at which gradient is computed.
        dx: slack for finite difference of partial derivatives.
        
    Output:
        (ndarray) of size domain of f.
    """
    x = x.reshape(1, -1)
    dim = x.shape[1]
    return np.array([deriv1(f, x, i, dx) for i in range(dim)]).reshape(1, -1)

def hessian(f ,x, dx=1e-6):
    """Computes hessian of f at point x.
    
    Args:
        f: objective function.
        x: point at which hessian is computed.
        dx: slack for finite difference of partial derivatives.
        
    Output:
        (ndarray) of square shape and size domain of f.
    """
    x = x.reshape(1, -1)
    dim = x.shape[1]
    line = lambda i: np.array([deriv2(f, x, i, j, dx) for j in range(dim)])
    return np.array([line(i) for i in range(dim)])

def condition_number(f, x):
    H = hessian(f, x)
    eivals, _ = np.linalg.eig(H)
    return abs(max(eivals) / min(eivals))

In [27]:
class ClassicUpdater:
    def __init__(self, rate=0.01, tol=1e-6):
        self.rate = rate
        self.tol = tol

    def __call__(self, f, x, reset):
        return self.rate * -gradient(f, x, self.tol)

class GD():
    """Gradient Descent Object.
    
    Implements gradient descent aiming to compute optimal objective 
    value of convex functions and local optimal ones of none 
    convex functions.
    """    
    def __init__(self, delta=None, decay=None, tol=1e-6, max_iter=1000):
        """
        Instantiates a GD object.
    
        Attributes:
        delta: function computing descent update vector (direction + step)
        decay: function computing decay
        tol: slack tolerance.
        max_iter: upper bound on number of iterations.
        """
        self.delta = delta if delta is not None else ClassicUpdater(0.01, tol)
        self.decay = decay if decay is not None else lambda f, x: np.linalg.norm(gradient(f, x, tol))
        self.tol = tol
        self.max_iter = max_iter
        self.grad = gradient
    
    def __call__(self, x, f):
        """Calling gradient descent object with specific starting point and optimal function.
        
        Args:
            x: initial starting point for descent.
            f: objective function of optimisation problem.
        
        Output:
            (float) sub-optimal value up to tolerance if execution is proper.
            (ndarray) list of gradient descent iterates.
        """
        # Helper functions
        compute_delta = lambda x, reset=False: self.delta(f, x, reset)
        compute_decay = lambda x: self.decay(f, x)
        
        # Start
        x = x.reshape(1, -1)
        n_iter = 0
        iters, iters_dir = x, compute_delta(x, reset=True)
        decay = compute_decay(x)
        while decay > self.tol and n_iter < self.max_iter:
            ## Decide on direction
            delta = compute_delta(x)
            ## Update iterate
            x = x + delta
            ## Store on-going data
            iters = np.vstack([iters, x])
            iters_dir = np.vstack([iters_dir, delta])
            ## Update decay
            decay = compute_decay(x)
            ## Update iteration number
            n_iter += 1

        # Display results
        msg = " Iteration nu. = {}\n approx. = {}\n ob value = {}\n and decay = {}."
        print(msg.format(n_iter, x.flatten(), f(x), decay))
        if decay > self.tol:
            warnings.warn("Decay didn't get under tolerance rate.", RuntimeWarning)
        return (x, iters, iters_dir, n_iter) 

In [35]:
DG_classic = GD()
DG_classic(np.r_[15, 20], ffamilies[0](0))
print()

 Iteration nu. = 878
 approx. = [2.96883265e-07 3.95844353e-07]
 ob value = 2.4483242466671736e-13
 and decay = 9.896108824517185e-07.



In [36]:
class NesterovUpdater:
    def __init__(self, rate=0.01, momentum=0.9, tol=1e-6):
        self.rate = rate
        self.momentum = momentum
        self.tol = tol
    
    def __call__(self, f, x, reset):
        x = x.reshape(1, -1)
        if reset is True:
            self.m_vect = np.zeros(x.shape[1])
        mv = self.momentum * self.m_vect
        self.m_vect = mv - self.rate * gradient(f, x + mv, self.tol)
        return self.m_vect

class AdamUpdater:
    def __init__(self, rate=0.01, beta1=0.9, beta2=0.999, tol=1e-6):
        self.rate = rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.tol = tol

    def __call__(self, f, x, reset):
        x = x.reshape(1, -1)
        if reset is True:
            self.m_vect = np.zeros(x.shape[1])
            self.s_vect = np.zeros(x.shape[1])
        gf = gradient(f, x, self.tol)
        m = self.beta1 * self.m_vect - (1 - self.beta1) * gf
        s = self.beta2 * self.s_vect + (1 - self.beta2) * np.cross(gf, gf)
        # TODO

In [37]:
DG_nesterov = GD(NesterovUpdater())
DG_nesterov(np.r_[15, 20], ffamilies[0](0))
DG_adam = GD(AdamUpdater())
DG_adam(np.r_[15, 20], ffamilies[0](0))
print()

 Iteration nu. = 252
 approx. = [3.17311626e-08 4.23082156e-08]
 ob value = 2.7968517893163512e-15
 and decay = 1.0577054011994732e-07.

